In [115]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.6.0 to work with ml-teaching-workspace


### Deploy a Model as a Web Service

In [116]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

xgboost-regression-hyperdrive version: 2
	 project : Regression
	 user : sifakhra


xgboost-regression-hyperdrive version: 1
	 project : Regression
	 user : sifakhra


AutoML1b9ab0dec61 version: 1




In [117]:
# Right, now let's get the model that we want to deploy. By default, if we specify a model name, the latest version will be returned.

model = ws.models['xgboost-regression-hyperdrive']
print(model.name, 'version', model.version)

xgboost-regression-hyperdrive version 2


In [118]:
# now let's create a web service to host this model, and this will require some code and configuration files; so let's create a folder for those.

import os

folder_name = 'auto_mpg_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

auto_mpg_service folder created.


The web service where we deploy the model will need some Python code to load the input data, get the model from the workspace, and generate and return predictions. We'll save this code in an entry script that will be deployed to the web service:

In [119]:
%%writefile $folder_name/score_auto_mpg.py
import json
import joblib
import numpy as np
from azureml.core.model import Model
from xgboost import XGBRegressor
import pandas as pd

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('xgboost-regression-hyperdrive')
#   model_path = Model.get_model_path('xgboost-regression-hyperdrive',2,ws)
    model = joblib.load(model_path)


# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    #data = json.loads(raw_data)['data']
    columns = ['Cylinders','Displacement','Horsepower','Weight','Acceleration','Model Year','Origin_is_Europe','Origin_is_Japan',
              'Origin_is_USA']
    df = pd.DataFrame(data, columns = columns)
    # Get a prediction from the model
    predictions = model.predict(df)   
    predictionList = []
    for prediction in predictions:
        predictionList.append(str(prediction))        

    # Return the predictions as JSON
    return json.dumps(predictionList)
    

Overwriting auto_mpg_service/score_auto_mpg.py


The web service will be hosted in a container, and the container will need to install any required Python dependencies when it gets initialized. In this case, our scoring code requires scikit-learn, so we'll create a .yml file that tells the container host to install this into the environment.

In [120]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("xgboost==0.90")
# Save the environment config as a .yml file
env_file = folder_name + "/auto_mpg_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in auto_mpg_service/auto_mpg_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
- xgboost==0.90
channels:
- anaconda
- conda-forge



Now you're ready to deploy. We'll deploy the container a service named diabetes-service. The deployment process includes the following steps:

Define an inference configuration, which includes the scoring and environment files required to load and use the model.
Define a deployment configuration that defines the execution environment in which the service will be hosted. In this case, an Azure Container Instance.
Deploy the model as a web service.
Verify the status of the deployed service.
More Information: For more details about model deployment, and options for target execution environments, see the [documentation](https://docs.microsoft.com/en-gb/azure/machine-learning/service/how-to-deploy-and-where).

Deployment will take some time as it first runs a process to create a container image, and then runs a process to create a web service based on the image. When deployment has completed successfully, you'll see a status of Healthy.

In [121]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_auto_mpg.py",
                                   conda_file="auto_mpg_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4)

service_name = "auto-mpg-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)



Running.........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [122]:
print(service.state)
print(service.get_logs())

Healthy
2020-06-11T16:02:43,232112178+00:00 - iot-server/run 
2020-06-11T16:02:43,233258234+00:00 - rsyslog/run 
2020-06-11T16:02:43,233366239+00:00 - gunicorn/run 
2020-06-11T16:02:43,233918966+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_9f19cf96fcb5fe9d16ffec08ab7f6760/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9f19cf96fcb5fe9d16ffec08ab7f6760/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9f19cf96fcb5fe9d16ffec08ab7f6760/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9f19cf96fcb5fe9d16ffec08ab7f6760/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9f19cf96fcb5fe9d16ffec08ab7f6760/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)


### Use the Web Service


In [129]:
import json


columns = ['Cylinders','Displacement','Horsepower','Weight','Acceleration','Model Year','Origin_is_Europe','Origin_is_Japan',
 'Origin_is_USA']
x_new = [[4,107,130,1504,12,70,0,0,1],[4,107,130,1504,12,70,0,0,1]]
print ('Auto: {}'.format(x_new[0]))


# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)
input_json

print(predictions)

Auto: [4, 107, 130, 1504, 12, 70, 0, 0, 1]
["25.591145", "25.591145"]


In [114]:
service.delete()